In [3]:
from create_nrPDB_GO_annot import read_fasta, load_clusters
from biotoolbox.structure_file_reader import build_structure_container_for_pdb
from biotoolbox.contact_map_builder import DistanceMapBuilder
from Bio.PDB import PDBList
import pandas as pd

from functools import partial
import numpy as np
import argparse
import csv
import os

def load_GO_annot(filename):
    """ Load GO annotations """
    onts = ['molecular_function', 'biological_process', 'cellular_component']
    prot2annot = {'EntryID':[],'term':[],'aspect':[]}
    goterms = {ont: [] for ont in onts}
    gonames = {ont: [] for ont in onts}
    with open(filename, mode='r') as tsvfile:
        reader = csv.reader(tsvfile, delimiter='\t')

        # molecular function
        next(reader, None)  # skip the headers
        goterms[onts[0]] = next(reader)
        next(reader, None)  # skip the headers
        gonames[onts[0]] = next(reader)
        onts[0] = 'MFO'
        # biological process
        next(reader, None)  # skip the headers
        goterms[onts[1]] = next(reader)
        next(reader, None)  # skip the headers
        gonames[onts[1]] = next(reader)
        onts[1] = 'BPO'
        # cellular component
        next(reader, None)  # skip the headers
        goterms[onts[2]] = next(reader)
        next(reader, None)  # skip the headers
        gonames[onts[2]] = next(reader)
        onts[2] = 'CCO'

        next(reader, None)  # skip the headers
        for row in reader:
            prot, prot_goterms = row[0], row[1:]
            for i in range(3):
                for goterm in prot_goterms[i].split(','):
                    if goterm != '':
                        prot2annot['EntryID'].append(prot)
                        prot2annot['term'].append(goterm)
                        prot2annot['aspect'].append(onts[i])
    return prot2annot

In [4]:
go_annot = pd.DataFrame(load_GO_annot('./data/CAFA1_annot.tsv'))
go_annot.to_csv('./data/CAFA1_train_terms.tsv',index=False,sep='\t')